In [ ]:
import psycopg2

DB_CONFIG = {
    'host': 'localhost',
    'port': 5433,
    'database': 'airflow',
    'user': 'airflow',
    'password': 'airflow'
}

conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()

cur.execute("""
    CREATE TEMP TABLE teste_escrita (
        id INT,
        descricao TEXT
    );
""")

cur.execute("""
    INSERT INTO teste_escrita VALUES (1, 'teste ok');
""")

cur.execute("SELECT * FROM teste_escrita;")
print(cur.fetchall())

conn.commit()
cur.close()
conn.close()
